In [5]:
!pip install pandas tqdm

Defaulting to user installation because normal site-packages is not writeable


In [58]:
import os
import tqdm
import shutil
import pandas as pd

In [59]:
DATASET_PATH = r"./dataset"
RAW_IMG_PATH = os.path.join(DATASET_PATH, "raw-img")

TRAIN_IMG_PATH = os.path.join(DATASET_PATH, 'train')
TEST_IMG_PATH = os.path.join(DATASET_PATH, 'test')
os.makedirs(TRAIN_IMG_PATH, exist_ok=True)
os.makedirs(TEST_IMG_PATH, exist_ok=True)

TEST_DATA_PER = 0.15

In [60]:
def collect_dataset(img_dir_path: str, imgs: list, target_path: str, group: str):
    annotation = {
      'filename': [],
      'label': [],
      'group': []
    }
    label = img_dir_path.split("/")[-1]
    for img in imgs:
        img_path = os.path.join(img_dir_path, img)
        new_img_path = os.path.join(target_path, img)
        shutil.copy(img_path, new_img_path)
        annotation['filename'].append(img)
        annotation['label'].append(label)
        annotation['group'].append(group)
    
    return annotation

def update_annotation(curr: dict, in_: dict):
    curr['filename'].extend(in_['filename'])
    curr['label'].extend(in_['label'])
    curr['group'].extend(in_['group'])

In [62]:
raw_img_category_dirs = os.listdir(RAW_IMG_PATH)
annotation = {
    'filename': [],
    'label': [],
    'group': []
}
for dir in tqdm.tqdm(raw_img_category_dirs):
    dir_path = os.path.join(RAW_IMG_PATH, dir)
    if os.path.isdir(dir_path):
        img_files = [f for f in os.listdir(dir_path) if f.lower().endswith(('.jpeg',))]
        boundary_idx = int(len(img_files) * (1 - TEST_DATA_PER))
        train_files, test_files = img_files[:boundary_idx], img_files[:boundary_idx]
        
        train_annotation = collect_dataset(dir_path, train_files, TRAIN_IMG_PATH, 'train')
        update_annotation(annotation, train_annotation)
        
        test_annotation = collect_dataset(dir_path, test_files, TEST_IMG_PATH, 'test')
        update_annotation(annotation, test_annotation)

annotation_df = pd.DataFrame(annotation)
annotation_df.to_csv(
    os.path.join(DATASET_PATH, 'annotations.csv'), index=True, index_label='index'
)

100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  5.94it/s]
